In [40]:
import pandas as pd
import numpy as np
import json
import itertools
from apyori import apriori  

In [41]:
df = pd.read_csv("../data/location1_cleaned.csv")
del df['product_sku']
del df['display_description']
del df['location_name']
df_prods = pd.read_csv("../data_engineering/products_clean.csv")
df_prods_idx = df_prods.copy().set_index('id')

def get_name(id):
    return df_prods_idx.loc[id,'name']

def get_cat(id):
    return df_prods_idx.loc[id,'category']

def get_name_cat(id):
    return (df_prods_idx.loc[id,'name'],df_prods_idx.loc[id,'category'])

In [42]:
#Categories:
cats = list(set(list(df_prods['category'])))
cats

['Beverage', 'Dessert', 'Entree', 'Side', 'Kids', 'Alcoholic Beverage']

In [45]:
# remove kids
#
# Here we are doing the following: We remove the kids line items from orders. 
# Back over with the kids recommendations we followed a different strategy.
# It's not clear what's best.
#
df = df[ df['item_category_type'] != 'Kids' ]

# get a list of unique order IDs
order_ids = list(set(list(df['order_id'])))
print(len(order_ids))

14163


In [47]:
# Here we build the 2D orders array that will be fed into apyori
#
# The idea is the following: for each order, we consider all possible combinations
# of entree,side,beverage,acoholic beverage, and dessert to handle the fact that often more than one of
# these categories is purchased. Based on the data, it's impossible to discern
# what belongs together, so we need to account for all possiblities. 
# Of course, this will reduce the predictive power of the outcome.
#

orders = []

# loop over all orders that fit our criteria
for i in range(len(order_ids)):
    
    # get all items for each order
    dd = (df.loc[ df['order_id'] == order_ids[i]]).copy()
    # drop duplicate items
    dd.drop_duplicates(subset='items_id',inplace=True)

    entree = list( dd.loc[ dd['item_category_type'] == 'Entree', 'items_id']  )
    nentree = len(entree)    
    dessert = list(dd.loc[ dd['item_category_type'] == 'Dessert', 'items_id'])
    ndessert = len(dessert)
    bev = list(dd.loc[ dd['item_category_type'] == 'Beverage','items_id'])
    nbev = len(bev)
    side = list(dd.loc[ dd['item_category_type'] == 'Side', 'items_id'])
    nside = len(side)
    alcohol = list(dd.loc[ dd['item_category_type'] == 'Alcoholic Beverage', 'items_id'])
    nalcohol = len(alcohol)
    
    #if i == 857:
    print(f"{i}, {order_ids[i]} nk: {entree} ns: {nside} nb: {nbev} nd: {ndessert}")

    
    # loop over all the orders stuff
    for j in range(nentree):
        this_order = []
        if nside == 0 and nbev == 0 and ndessert == 0 and nalcohol == 0:
            this_order.append( [ entree[j] ] )
            continue

        if nside > 0:
            if nalcohol > 0:
                if nbev == 0 and ndessert == 0:
                    xx = list(itertools.product(side,alcohol))
                if nbev > 0 and ndessert == 0:
                    xx = list(itertools.product(side,alcohol,bev))
                if nbev == 0 and ndessert > 0:
                    xx = list(itertools.product(side,alcohol,dessert))
                if nbev > 0 and ndessert > 0:
                    xx = list(itertools.product(side,alcohol,dessert,bev))
            else:
                if nbev == 0 and ndessert == 0:
                    xx = list(itertools.combinations(side,1))
                if nbev > 0 and ndessert == 0:
                    xx = list(itertools.product(side,bev))
                if nbev == 0 and ndessert > 0:
                    xx = list(itertools.product(side,dessert))
                if nbev > 0 and ndessert > 0:
                    xx = list(itertools.product(side,dessert,bev))                                    
        else:
            if nalcohol > 0:
                if nbev == 0 and ndessert == 0:
                   xx = list(itertools.combinations(alcohol,1))
                if nbev > 0 and ndessert == 0:
                    xx = list(itertools.product(bev,alcohol))
                if nbev == 0 and ndessert > 0:
                    xx = list(itertools.product(dessert,alcohol))
                if nbev > 0 and ndessert > 0:
                    xx = list(itertools.product(bev,dessert,alcohol))
            else:
                if nbev == 0 and ndessert == 0:
                   assert(False) # this should never be reached
                if nbev > 0 and ndessert == 0:
                    xx = list(itertools.combinations(bev,1))
                if nbev == 0 and ndessert > 0:
                    xx = list(itertools.combinations(dessert,1))
                if nbev > 0 and ndessert > 0:
                    xx = list(itertools.product(bev,dessert))                
            
        for k in range(len(xx)):
            temp_order = []
            temp_order.append(entree[j])
            for l in range(len(xx[k])):
                temp_order.append(xx[k][l])
            this_order.append(temp_order)

    print(this_order)
    orders += this_order
    


0, 1048576 nk: [2237, 3036] ns: 0 nb: 0 nd: 0
[[3036, 3825]]
1, 884737 nk: [] ns: 0 nb: 1 nd: 0
[[3036, 3825]]
2, 851970 nk: [2210, 3037] ns: 0 nb: 1 nd: 0
[[3037, 1858]]
3, 884739 nk: [2238, 2716] ns: 1 nb: 1 nd: 1
[[2716, 1813, 2245, 2218]]
4, 917508 nk: [2307, 2238] ns: 0 nb: 1 nd: 1
[[2238, 1858, 2208]]
5, 950277 nk: [2716] ns: 1 nb: 1 nd: 0
[[2716, 1813, 1858]]
6, 1015810 nk: [3265] ns: 0 nb: 0 nd: 0
[[3265]]
7, 1015813 nk: [3285] ns: 0 nb: 1 nd: 0
[[3285, 1858]]
8, 1081349 nk: [2716] ns: 0 nb: 1 nd: 0
[[2716, 1858]]
9, 851981 nk: [2716, 2237, 3265, 2210] ns: 0 nb: 1 nd: 0
[[2210, 2218]]
10, 950285 nk: [2210] ns: 0 nb: 1 nd: 0
[[2210, 2218]]
11, 1048591 nk: [] ns: 0 nb: 1 nd: 0
[[2210, 2218]]
12, 851988 nk: [2210, 2716, 3038] ns: 0 nb: 3 nd: 0
[[3038, 2218], [3038, 1852], [3038, 1858]]
13, 917524 nk: [] ns: 0 nb: 1 nd: 1
[[3038, 2218], [3038, 1852], [3038, 1858]]
14, 1015835 nk: [3288] ns: 0 nb: 0 nd: 0
[[3288]]
15, 950300 nk: [2210, 2238, 3036] ns: 0 nb: 1 nd: 0
[[3036, 1858]]
16

151, 950596 nk: [] ns: 0 nb: 1 nd: 0
[[2210, 1858]]
152, 983368 nk: [2237, 2210, 3038] ns: 0 nb: 1 nd: 0
[[3038, 1858]]
153, 950602 nk: [3037, 3288, 2210] ns: 0 nb: 0 nd: 0
[[2210]]
154, 950603 nk: [2237, 3288] ns: 0 nb: 0 nd: 0
[[3288, 3825]]
155, 1048908 nk: [2237, 3285] ns: 1 nb: 1 nd: 0
[[3285, 3289, 2218]]
156, 885069 nk: [] ns: 0 nb: 0 nd: 1
[[3285, 3289, 2218]]
157, 917837 nk: [2716, 2210] ns: 0 nb: 1 nd: 0
[[2210, 1858]]
158, 852303 nk: [2238] ns: 1 nb: 0 nd: 0
[[2238, 1813, 2256]]
159, 885071 nk: [] ns: 0 nb: 1 nd: 0
[[2238, 1813, 2256]]
160, 917839 nk: [2237, 3038, 2716] ns: 0 nb: 1 nd: 0
[[2716, 1858]]
161, 983377 nk: [3288] ns: 0 nb: 1 nd: 0
[[3288, 1858]]
162, 1114451 nk: [2210, 2716] ns: 0 nb: 1 nd: 0
[[2716, 1858]]
163, 983382 nk: [2210, 2716] ns: 0 nb: 2 nd: 0
[[2716, 2218], [2716, 1858]]
164, 1114454 nk: [3037, 2210] ns: 0 nb: 0 nd: 0
[[2210]]
165, 885080 nk: [] ns: 0 nb: 0 nd: 0
[[2210]]
166, 852315 nk: [2716, 3038, 2210] ns: 0 nb: 3 nd: 0
[[2210, 1852], [2210, 2218],

305, 950970 nk: [2238, 2210] ns: 0 nb: 1 nd: 0
[[2210, 1858]]
306, 950978 nk: [2237] ns: 0 nb: 1 nd: 0
[[2237, 1858, 2255]]
307, 950979 nk: [3037, 3038] ns: 0 nb: 1 nd: 0
[[3038, 1858]]
308, 852677 nk: [3265, 3389] ns: 0 nb: 1 nd: 0
[[3389, 1858]]
309, 983750 nk: [2307, 2210] ns: 1 nb: 1 nd: 0
[[2210, 1838, 1858]]
310, 918221 nk: [3288] ns: 0 nb: 0 nd: 0
[[3288, 3039]]
311, 983758 nk: [] ns: 1 nb: 2 nd: 0
[[3288, 3039]]
312, 950992 nk: [2237, 2716, 2307, 2238] ns: 0 nb: 2 nd: 0
[[2238, 1852], [2238, 1858]]
313, 950996 nk: [2716, 2237] ns: 0 nb: 1 nd: 0
[[2237, 1858, 2255]]
314, 918231 nk: [] ns: 1 nb: 0 nd: 0
[[2237, 1858, 2255]]
315, 951000 nk: [2716] ns: 0 nb: 0 nd: 0
[[2716]]
316, 951002 nk: [2237, 2238] ns: 0 nb: 2 nd: 0
[[2238, 2251], [2238, 1858]]
317, 951003 nk: [2210] ns: 0 nb: 1 nd: 0
[[2210, 2218]]
318, 885471 nk: [3038] ns: 0 nb: 0 nd: 0
[[3038]]
319, 951009 nk: [2307, 2238] ns: 0 nb: 2 nd: 0
[[2238, 1858], [2238, 2218]]
320, 951012 nk: [2716, 3288] ns: 0 nb: 2 nd: 0
[[3288,

439, 1115175 nk: [2236, 2237, 2307] ns: 0 nb: 0 nd: 1
[[2307, 3411]]
440, 1115176 nk: [2237, 3036] ns: 0 nb: 2 nd: 0
[[3036, 2251], [3036, 1858]]
441, 1049641 nk: [2210, 3036] ns: 0 nb: 0 nd: 0
[[3036]]
442, 853034 nk: [3288] ns: 0 nb: 0 nd: 0
[[3288]]
443, 1016874 nk: [2238] ns: 0 nb: 0 nd: 0
[[2238]]
444, 1049645 nk: [2716, 3038, 2238] ns: 0 nb: 1 nd: 0
[[2238, 1858]]
445, 951346 nk: [3265] ns: 0 nb: 0 nd: 0
[[3265]]
446, 1115186 nk: [2716, 3036] ns: 0 nb: 1 nd: 0
[[3036, 1858]]
447, 951348 nk: [3038] ns: 0 nb: 0 nd: 0
[[3038, 2256], [3038, 3825]]
448, 1115188 nk: [2238, 2210, 2716] ns: 0 nb: 0 nd: 0
[[2716]]
449, 918585 nk: [3265] ns: 0 nb: 0 nd: 0
[[3265]]
450, 1115194 nk: [2238] ns: 0 nb: 1 nd: 0
[[2238, 1858]]
451, 885819 nk: [2237, 3370] ns: 0 nb: 2 nd: 0
[[3370, 1858], [3370, 2218]]
452, 918587 nk: [3285] ns: 0 nb: 0 nd: 0
[[3285]]
453, 918588 nk: [2237] ns: 0 nb: 0 nd: 0
[[2237]]
454, 918590 nk: [3285] ns: 0 nb: 2 nd: 0
[[3285, 2218], [3285, 1858]]
455, 1016895 nk: [2238] ns: 

604, 1050065 nk: [2237] ns: 0 nb: 0 nd: 0
[[2237, 3039]]
605, 1050076 nk: [2210, 3038] ns: 0 nb: 0 nd: 0
[[3038, 2256]]
606, 1017313 nk: [2237, 3288] ns: 0 nb: 2 nd: 0
[[3288, 2218], [3288, 1858]]
607, 1082851 nk: [] ns: 0 nb: 1 nd: 0
[[3288, 2218], [3288, 1858]]
608, 919020 nk: [2237, 3285] ns: 0 nb: 0 nd: 0
[[3285]]
609, 1050092 nk: [2716] ns: 0 nb: 1 nd: 0
[[2716, 1858]]
610, 1050099 nk: [3265, 2236, 2210] ns: 0 nb: 0 nd: 0
[[2210]]
611, 853500 nk: [] ns: 0 nb: 1 nd: 0
[[2210]]
612, 1050109 nk: [2716] ns: 0 nb: 0 nd: 0
[[2716]]
613, 919040 nk: [3285] ns: 0 nb: 1 nd: 0
[[3285, 1858]]
614, 853505 nk: [3370, 3285, 3389] ns: 0 nb: 0 nd: 0
[[3389]]
615, 919045 nk: [3285, 3036] ns: 0 nb: 1 nd: 0
[[3036, 1858]]
616, 1017350 nk: [2210, 2238, 2307] ns: 0 nb: 0 nd: 0
[[2307]]
617, 1017351 nk: [2236, 3038] ns: 0 nb: 0 nd: 0
[[3038]]
618, 1017360 nk: [] ns: 0 nb: 0 nd: 0
[[3038]]
619, 1082900 nk: [2237, 2238] ns: 0 nb: 0 nd: 0
[[2238, 2256]]
620, 984607 nk: [3370, 3285] ns: 0 nb: 2 nd: 0
[[3285

...

13504, 883244 nk: [2716] ns: 1 nb: 1 nd: 0
[[2716, 1813, 1852]]
13505, 916014 nk: [2210, 3265, 2238, 2716] ns: 0 nb: 0 nd: 0
[[2716]]
13506, 916015 nk: [3037, 2716] ns: 0 nb: 1 nd: 0
[[2716, 1852]]
13507, 948782 nk: [2210] ns: 0 nb: 1 nd: 0
[[2210, 1858]]
13508, 916019 nk: [2210, 3288] ns: 0 nb: 1 nd: 0
[[3288, 1855, 3825]]
13509, 916022 nk: [2210] ns: 0 nb: 1 nd: 0
[[2210, 1858]]
13510, 948791 nk: [] ns: 0 nb: 1 nd: 0
[[2210, 1858]]
13511, 883260 nk: [2210] ns: 0 nb: 0 nd: 0
[[2210, 2255]]
13512, 916028 nk: [3038, 2210] ns: 0 nb: 1 nd: 0
[[2210, 1858]]
13513, 948796 nk: [2307, 2238, 2237] ns: 0 nb: 2 nd: 0
[[2237, 1858], [2237, 1852]]
13514, 948797 nk: [2210] ns: 0 nb: 0 nd: 0
[[2210]]
13515, 948801 nk: [] ns: 0 nb: 2 nd: 0
[[2210]]
13516, 916034 nk: [2238] ns: 0 nb: 1 nd: 0
[[2238, 2218]]
13517, 916037 nk: [3288, 3265] ns: 0 nb: 1 nd: 0
[[3265, 1858]]
13518, 850502 nk: [] ns: 0 nb: 1 nd: 0
[[3265, 1858]]
13519, 981574 nk: [2237, 3288] ns: 0 nb: 1 nd: 0
[[3288, 2218, 3825]]
13520, 883

13634, 916309 nk: [2210] ns: 0 nb: 1 nd: 0
[[2210, 1852]]
13635, 916310 nk: [2210, 2716] ns: 0 nb: 0 nd: 0
[[2716]]
13636, 1112920 nk: [3037, 2238] ns: 0 nb: 1 nd: 0
[[2238, 1858]]
13637, 981849 nk: [2237, 3370] ns: 0 nb: 1 nd: 0
[[3370, 1858]]
13638, 916315 nk: [2716, 2238, 2237] ns: 0 nb: 1 nd: 0
[[2237, 1858]]
13639, 883550 nk: [2210] ns: 0 nb: 2 nd: 0
[[2210, 1858], [2210, 1854]]
13640, 883552 nk: [3036, 2307] ns: 0 nb: 1 nd: 0
[[2307, 1852]]
13641, 883553 nk: [2210, 2238] ns: 0 nb: 1 nd: 0
[[2238, 1858]]
13642, 850786 nk: [3370, 3285] ns: 0 nb: 0 nd: 0
[[3285]]
13643, 1014625 nk: [2238] ns: 0 nb: 1 nd: 0
[[2238, 2218]]
13644, 1112930 nk: [2307, 3288] ns: 0 nb: 1 nd: 0
[[3288, 1858, 2255]]
13645, 883560 nk: [2210] ns: 0 nb: 1 nd: 0
[[2210, 1858]]
13646, 1080170 nk: [3038, 3288] ns: 0 nb: 0 nd: 0
[[3288]]
13647, 1014639 nk: [2237, 2210] ns: 0 nb: 0 nd: 0
[[2210]]
13648, 1112943 nk: [3038, 2237] ns: 0 nb: 0 nd: 0
[[2237, 2256], [2237, 3039]]
13649, 916338 nk: [3265, 2307] ns: 0 nb: 0

13793, 1113225 nk: [2307, 3037] ns: 0 nb: 1 nd: 0
[[3037, 1858, 2256]]
13794, 851084 nk: [2237, 3288] ns: 0 nb: 0 nd: 0
[[3288]]
13795, 1047693 nk: [3037] ns: 0 nb: 1 nd: 0
[[3037, 1858]]
13796, 1113230 nk: [] ns: 0 nb: 1 nd: 0
[[3037, 1858]]
13797, 982162 nk: [3389, 3285] ns: 0 nb: 0 nd: 0
[[3285, 2255], [3285, 3039]]
13798, 883860 nk: [2307] ns: 0 nb: 1 nd: 1
[[2307, 1858, 3411]]
13799, 851097 nk: [3285] ns: 3 nb: 1 nd: 0
[[3285, 2234, 1858], [3285, 2221, 1858], [3285, 3289, 1858]]
13800, 883865 nk: [3288, 2210, 3038] ns: 0 nb: 1 nd: 0
[[3038, 1858]]
13801, 883867 nk: [2238, 2237] ns: 0 nb: 1 nd: 0
[[2237, 1858]]
13802, 1113246 nk: [3288, 3037, 3036] ns: 0 nb: 1 nd: 0
[[3036, 1858]]
13803, 1047711 nk: [3038, 3288, 2238, 2210] ns: 0 nb: 3 nd: 0
[[2210, 1858], [2210, 1855], [2210, 1852]]
13804, 1080484 nk: [2210, 3288] ns: 0 nb: 1 nd: 1
[[3288, 1858, 3411]]
13805, 1047717 nk: [3036, 3288, 3038] ns: 0 nb: 1 nd: 0
[[3038, 1858, 3825]]
13806, 949414 nk: [] ns: 1 nb: 0 nd: 0
[[3038, 1858, 

13922, 1015291 nk: [2210, 3288] ns: 0 nb: 1 nd: 0
[[3288, 1858]]
13923, 916991 nk: [3288] ns: 1 nb: 1 nd: 1
[[3288, 1838, 2208, 2218]]
13924, 1080831 nk: [2237, 3038] ns: 0 nb: 1 nd: 0
[[3038, 1858]]
13925, 1048065 nk: [3288] ns: 0 nb: 1 nd: 0
[[3288, 1852]]
13926, 851459 nk: [] ns: 0 nb: 0 nd: 0
[[3288, 1852]]
13927, 1048069 nk: [2210] ns: 0 nb: 1 nd: 0
[[2210, 1858]]
13928, 884230 nk: [2237] ns: 0 nb: 0 nd: 0
[[2237]]
13929, 916999 nk: [2716] ns: 1 nb: 2 nd: 0
[[2716, 1813, 1856], [2716, 1813, 1858]]
13930, 1015309 nk: [3038] ns: 0 nb: 0 nd: 0
[[3038]]
13931, 1048080 nk: [3265, 3288, 2238] ns: 0 nb: 1 nd: 0
[[2238, 1858]]
13932, 884241 nk: [2210, 2237] ns: 0 nb: 1 nd: 0
[[2237, 1858]]
13933, 917009 nk: [2237] ns: 0 nb: 0 nd: 0
[[2237]]
13934, 1048081 nk: [2716, 2210] ns: 0 nb: 2 nd: 0
[[2210, 1858], [2210, 2251]]
13935, 1048087 nk: [] ns: 0 nb: 1 nd: 0
[[2210, 1858], [2210, 2251]]
13936, 1048090 nk: [2237] ns: 0 nb: 1 nd: 0
[[2237, 1858]]
13937, 917024 nk: [2210, 3037] ns: 0 nb: 1 nd

14046, 851686 nk: [3038, 3288] ns: 0 nb: 0 nd: 0
[[3288, 3039], [3288, 2294]]
14047, 1048298 nk: [2716, 2237, 2210] ns: 0 nb: 2 nd: 0
[[2210, 1858, 3825], [2210, 1858, 2255], [2210, 2218, 3825], [2210, 2218, 2255]]
14048, 917227 nk: [2238] ns: 0 nb: 1 nd: 0
[[2238, 1858]]
14049, 1015531 nk: [3037] ns: 0 nb: 0 nd: 0
[[3037]]
14050, 884464 nk: [2210] ns: 0 nb: 1 nd: 0
[[2210, 1858]]
14051, 1015538 nk: [3038] ns: 0 nb: 0 nd: 0
[[3038, 2255]]
14052, 1048307 nk: [3038] ns: 0 nb: 1 nd: 0
[[3038, 2218]]
14053, 1048311 nk: [2237] ns: 0 nb: 0 nd: 0
[[2237]]
14054, 884472 nk: [] ns: 0 nb: 2 nd: 0
[[2237]]
14055, 1015544 nk: [3037] ns: 0 nb: 1 nd: 0
[[3037, 1852]]
14056, 851706 nk: [2716] ns: 0 nb: 1 nd: 0
[[2716, 1858]]
14057, 884477 nk: [2237, 2210] ns: 0 nb: 1 nd: 0
[[2210, 2218]]
14058, 950014 nk: [3038, 2210] ns: 0 nb: 0 nd: 0
[[2210, 2256], [2210, 3825]]
14059, 917247 nk: [3038] ns: 0 nb: 1 nd: 0
[[3038, 1858]]
14060, 982784 nk: [2307, 2716] ns: 0 nb: 0 nd: 0
[[2716]]
14061, 917249 nk: [328

In [48]:
print(len(orders))

17473


In [61]:
# here's an example calculation of a pair's support
count = 0
for i in range(len(orders)):
    if 3036 in orders[i] and 1858 in orders[i]:
        count += 1
        
print(count)
print(count/len(orders))

251
0.014365020317060608


In [80]:
# run the apriori algorithm using apyori
# Since we have relativiely few data sets, we only consider pairs {X} -> {Y}
# We limit lift to > 1.05. The min_support and min_confidence levels are chosen
# to get at least a few cases with multiple possible combinations. We then
# pick the one with the greatest product of support * confidence * lift.

association_rules = apriori(orders, min_support=0.0001, min_confidence=0.06, min_lift=1.05, max_length=2)  
assoc = list(association_rules)  

In [81]:
print(len(assoc)) 

196


In [82]:
# messy
for item in assoc:
    if 3036 in list(item.items) and 1858 in list(item.items):
        print(item)

RelationRecord(items=frozenset({1858, 3036}), support=0.014365020317060608, ordered_statistics=[OrderedStatistic(items_base=frozenset({3036}), items_add=frozenset({1858}), confidence=0.48737864077669907, lift=1.0548701833632186)])


In [83]:
# get products sans Kids and alcohol
df_sa_prods = df_prods.loc[ (df_prods['category'] != 'Kids'), ['id','name','category']].copy()

df_sa_prods.sort_values('category')

,id,name,category
27,2294,Woodbridge Cabernet,Alcoholic Beverage
47,3825,Watermelon Margarita,Alcoholic Beverage
34,3039,Happy Daze,Alcoholic Beverage
26,2293,Woodbridge Chardonnay,Alcoholic Beverage
25,2257,Beso Del Sol Sangria,Alcoholic Beverage
24,2256,Yuengling,Alcoholic Beverage
23,2255,Bud Light,Alcoholic Beverage
54,5446,Winterberry Margarita,Alcoholic Beverage
8,1858,Fountain Beverage,Beverage
4,1852,Bottled Water,Beverage


In [88]:
# build the recommendation dict of dicts for Entrees
prodlist = df_sa_prods.T.to_dict().values()

reco_dict = {}
cats = ['Entree','Side','Dessert','Beverage','Alcoholic Beverage']


for prod in prodlist:
    print(prod)
    md = {}
    id = prod['id']
    md['category'] = prod['category']
    md[prod['category']] = -1
    md['Kids'] = -1

    search_cats = [ x for x in cats if x != prod['category'] ]
    print(search_cats)
    print(id)
    
    dd = [ x for x in assoc if list(x.items)[0] == id or list(x.items)[1] == id  ]
    # for each category, we take the pairing with the highest
    # support, since we have already placed a good limit on lift
    # If there is no pairing, we set that category to -2
    for cat in search_cats:
        
        xl = []
        for x in dd:
            for y in x.ordered_statistics:
                xd = {}
                xid = list(y.items_add)[0]
                if xid != id:
                    xd['id'] = xid
                    xd['support'] = x.support
                    xd['confidence'] = y.confidence
                    xd['lift'] = y.lift
                    if get_cat(xid) == cat:
                        xl.append(xd)    


        
        xs = sorted(xl,key = lambda x: x['support']*x['confidence']*x['lift'],reverse=True)
        if len(xs) > 0:
            md[cat] = xs[0]
        else:
            md[cat] = -2
        if prod['category'] == 'Entree':
            print(cat,xs)
        
        print('**')

    print('------------------')
        
    
    
    reco_dict[prod['id']] = md
    
    
    

{'id': 1813, 'name': 'French Fries', 'category': 'Side'}
['Entree', 'Dessert', 'Beverage', 'Alcoholic Beverage']
1813
**
**
**
**
------------------
{'id': 1838, 'name': 'Green Beans', 'category': 'Side'}
['Entree', 'Dessert', 'Beverage', 'Alcoholic Beverage']
1838
**
**
**
**
------------------
{'id': 1852, 'name': 'Bottled Water', 'category': 'Beverage'}
['Entree', 'Side', 'Dessert', 'Alcoholic Beverage']
1852
**
**
**
**
------------------
{'id': 1854, 'name': 'Hot Tea', 'category': 'Beverage'}
['Entree', 'Side', 'Dessert', 'Alcoholic Beverage']
1854
**
**
**
**
------------------
{'id': 1855, 'name': 'Chocolate Milk', 'category': 'Beverage'}
['Entree', 'Side', 'Dessert', 'Alcoholic Beverage']
1855
**
**
**
**
------------------
{'id': 1856, 'name': 'Lowfat Milk', 'category': 'Beverage'}
['Entree', 'Side', 'Dessert', 'Alcoholic Beverage']
1856
**
**
**
**
------------------
{'id': 1858, 'name': 'Fountain Beverage', 'category': 'Beverage'}
['Entree', 'Side', 'Dessert', 'Alcoholic Bev

In [89]:
print(json.dumps(reco_dict,indent=2))

{
  "1813": {
    "category": "Side",
    "Side": -1,
    "Kids": -1,
    "Entree": {
      "id": 2716,
      "support": 0.005665884507525897,
      "confidence": 0.20886075949367086,
      "lift": 1.1930121120081434
    },
    "Dessert": -2,
    "Beverage": {
      "id": 1858,
      "support": 0.014708407256910663,
      "confidence": 0.5421940928270041,
      "lift": 1.1735113816383307
    },
    "Alcoholic Beverage": -2
  },
  "1838": {
    "category": "Side",
    "Side": -1,
    "Kids": -1,
    "Entree": {
      "id": 3288,
      "support": 0.0011446231328335146,
      "confidence": 0.15503875968992248,
      "lift": 1.3544961240310076
    },
    "Dessert": {
      "id": 3411,
      "support": 0.0005723115664167573,
      "confidence": 0.07751937984496124,
      "lift": 1.7080657301778157
    },
    "Beverage": {
      "id": 1852,
      "support": 0.0009156985062668116,
      "confidence": 0.12403100775193798,
      "lift": 1.2426569945238604
    },
    "Alcoholic Beverage": -2
  }

In [90]:
# re-key dictionary so that it fits with what we have
# in the flask app
for key in reco_dict.keys():
    xlist = list(reco_dict[key].keys())
    for xkey in xlist:
        xx = reco_dict[key][xkey]
        if xkey == 'Alcoholic Beverage':
            reco_dict[key]['alcohol'] = xx
            del reco_dict[key][xkey]
        else:
            new_key = xkey.lower()
            reco_dict[key][new_key] = xx
            del reco_dict[key][xkey]


In [91]:
with open("../data/adults_recommendations.json",'w') as f:
    f.write(json.dumps(reco_dict,indent=2))